# SQL query from table names - Continued

In [24]:
!pip install python-dotenv
!pip install jupyter_bokeh

In [25]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [26]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})




# Function to generate SQL query based on the user's message (old context)
def return_old_CCRMSQL(user_message, old_context):
    client = OpenAI(api_key=OPENAI_API_KEY)

    newcontext = old_context.copy()
    newcontext.append({'role': 'user', 'content': "question: " + user_message})

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=newcontext,
        temperature=0,
    )

    return response.choices[0].message.content



## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [27]:
context = [
    {'role': 'system', 'content': """
        CREATE TABLE employees (
            ID_usr INT PRIMARY KEY,
            name VARCHAR(100)
        );

        INSERT INTO employees (ID_usr, name) VALUES (1, 'John Doe');
        INSERT INTO employees (ID_usr, name) VALUES (2, 'Jane Smith');

        CREATE TABLE salary (
            ID_usr INT,
            year DATE,
            salary FLOAT,
            FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
        );

        INSERT INTO salary (ID_usr, year, salary) VALUES (1, '2024-01-01', 50000);
        INSERT INTO salary (ID_usr, year, salary) VALUES (2, '2024-01-01', 60000);

        CREATE TABLE studies (
            ID INT PRIMARY KEY,
            ID_usr INT,
            educational_level INT,
            Institution VARCHAR(100),
            Years DATE,
            Speciality VARCHAR(100),
            FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
        );

        INSERT INTO studies (ID, ID_usr, educational_level, Institution, Years, Speciality)
        VALUES (1, 1, 2, 'Ohio University', '2020-05-01', 'Computer Science');
        INSERT INTO studies (ID, ID_usr, educational_level, Institution, Years, Speciality)
        VALUES (2, 2, 3, 'Harvard University', '2018-05-01', 'Business Administration');
    """}
]

context.append({'role': 'system', 'content': """
    -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.

    -- Example 1: Get all employees who have a salary greater than 55000.
    -- SQL:
    SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    WHERE s.salary > 55000;

    -- Example 2: List the name and educational level of employees who studied at Ohio University.
    -- SQL:
    SELECT e.name, st.educational_level
    FROM employees e
    JOIN studies st ON e.ID_usr = st.ID_usr
    WHERE st.Institution = 'Ohio University';

    -- Example 3: Find all employees who have a salary greater than 50000 and have a degree from 'Harvard University'.
    -- SQL:
    SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    JOIN studies st ON e.ID_usr = st.ID_usr
    WHERE s.salary > 50000 AND st.Institution = 'Harvard University';
"""})

# Function to call the model
def return_CCRMSQL(user_message, context):
    client = OpenAI(api_key=OPENAI_API_KEY)

    newcontext = context.copy()
    newcontext.append({'role': 'user', 'content': "question: " + user_message})

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=newcontext,
        temperature=0,
    )

    return response.choices[0].message.content


## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [28]:
#new
context_user = context.copy()
print(return_CCRMSQL("Get the highest salary employee", context_user))



```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```


In [29]:

old_context_user = old_context.copy()
print(return_old_CCRMSQL("Get the highest salary employee", old_context_user))


This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name and salary of the employee with the highest salary by joining the "employees" and "salary" tables on the ID_usr column. It then orders the results by salary in descending order and limits the output to only the first row, which represents the employee with the highest salary.


In [32]:
#new
print(return_CCRMSQL("Get the institution with the highest average salary", context_user))


```sql
SELECT st.Institution, AVG(s.salary) AS avg_salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```


In [33]:
#old
print(return_old_CCRMSQL("Get the institution with the highest average salary", old_context_user))


This is your SQL:
```sql
SELECT s.Institution, AVG(s.salary) AS avg_salary
FROM salary s
JOIN employees e ON s.ID_usr = e.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query retrieves the institution with the highest average salary by joining the `salary` table with the `employees` table on the `ID_usr` column. It calculates the average salary for each institution, orders the results in descending order based on the average salary, and then limits the output to only the top result.


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

       - Directive Clarity: Specific directives generate accurate SQL queries.

      - Dealing with Complexity: GPT can handle moderately complex queries, such as aggregations and joins.

      - Database Context: Proper context is essential for generating accurate SQL queries.

In [34]:
# Define the context for generating SQL queries
context_user = context.copy()

# Version 1: Query to find the employee with the highest salary
print(return_CCRMSQL("Get the employee with the highest salary", context_user))

# Version 2: Query to find the institution with the highest average salary
print(return_CCRMSQL("Get the institution with the highest average salary", context_user))

# Version 3: Query to get employees earning more than $50,000 with a degree in Computer Science
print(return_CCRMSQL("Get employees earning more than $50,000 and have a degree in Computer Science", context_user))


```sql
SELECT e.name 
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```
```sql
SELECT st.Institution, AVG(s.salary) AS avg_salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE s.salary > 50000 AND st.Speciality = 'Computer Science';
